# CFG to CNF (v7)

In [1]:
import re
from itertools import combinations
import copy

In [2]:
class CFG:
    def __init__(self, inp, vt, vn):
        self.inp = inp 
        self.vt = vt 
        self.vn = vn 
        self.cfg = {}
        self.cnf = {}

    # function to parse the input
    def parse_input(self):
        
        # splittig transitions when -> encountered and building a dict
        for transition in self.inp:
            transition = re.sub(r"\s+", "", transition)
            parts = transition.split("->")
            self.cfg[parts[0]] = parts[1].split("|")
        
        return self.cfg
    
    # function to convert CFG to CFG to CNF
    def cfg_2_cnf(self):
            
        # step 1 - adding s0 if start symbol on the RHS
        def check_start(): 
            cfg_1 = self.cfg.copy()

            for key in self.cfg.keys():
                for prod in self.cfg[key]:

                    # if S on the RHS add S0
                    if 'S' in prod:
                        cfg_1["S0"] = ["S"]
                        break
                        
            self.cfg = cfg_1
            
            return self.cfg
        
        # step 2 - remove null productions
        def get_combinations(string):
            
            # step 2.1 - obtaining all possible combinations
            
            sample_list = list(string)
            list_combinations = list()

            for n in range(len(sample_list) + 1):
                list_combinations += list(combinations(sample_list, n))

            return list_combinations
        
        
        def allCharactersNull(s, symbol):

            # step 2.2 - checking if a formed string contains only null characters

            if s[0] != symbol:
                return False

            for i in range(1, len(s)):
                if s[i] != s[0]:
                    return False

            return True
        
        def string_comb(list_combinations, symbol, prod):
            
            # transforming the obtained combnations to strings

            res = set()

            for i in range (len(list_combinations)):
                
                # forming a string of all characters from a substring
                mystr = ''.join([str(item) for item in list_combinations[i]])

                # if length = 1 and symbol different from the one that can be null
                if len(list_combinations[i]) == 1 and not symbol in list_combinations[i]:
                    res.add(mystr)
                    
                # if the string is different from the initial production
                elif mystr != prod and mystr != symbol and len(mystr) > 0:
                    res.add(mystr)
                
                # trnsforming the result to list because it is iterable
                r = list(res)
                combinations = list()
                
                # checking if a string has all characters the same which derive in null
                for i in range (len(r)):
                    if not allCharactersNull(r[i], symbol):
                        combinations.append(r[i])

            return combinations
        
        def remove_null(cfg):
            
            # removing the null productions
    
            cfg_2 = cfg.copy()

            null = False
            for key in cfg_2.keys():
                if "eps" in cfg_2[key]:

                    null = True
                    cfg_2[key].remove("eps")

                    # append eps
                    for k in cfg_2.keys():
                        if key in cfg_2[k]:
                            cfg_2[k].append("eps")
                            null = True

                    # append combinatons with eps
                    for k in cfg_2.keys():
                        for prod in cfg_2[k]:
                            if key in prod and len(prod) > 1:
                                list_combinations = get_combinations(prod)
                                add_comb = string_comb(list_combinations, key, prod)

                                [cfg_2[k].append(x) for x in add_comb if x not in cfg_2[k] and x in prod]

                    if null:
                        remove_null(cfg_2)
                        
            self.cfg = cfg_2
            return self.cfg
        
        # step 3 - remove unit productions
        def remove_unit(cfg):
    
            cfg_3 = cfg.copy()

            unit = False
            for key in cfg_3.keys():
                for prod in cfg_3[key]:
                    if prod in self.vn:
                        
                        if key == prod:
                            # if the key is the same as the unit production
                            cfg_3[key].remove(prod)
                        else:
                            # if the key is different from the production
                            cfg_3[key].remove(prod)
                            cfg_3[key] += cfg_3[prod]
                            unit = True

                        if unit:
                            remove_unit(cfg_3)
                            
            self.cfg = cfg_3.copy()
            return cfg_3
        
        # step 4 - remove productions where are more than 2 vriables
        def get_key(val, my_dict):
            # function to find he key of a value from the dict
            for key, value in my_dict.items():
                 if val == value:
                    return key

            return "key doesn't exist"
        
        def remove_long_prod():
            # removing cases where a production consists of 3+ productions
            
            i = 0
            replaced = {}
            cfg_4 = self.cfg.copy()
            
            for key in self.cfg.keys():
                for prod in self.cfg[key]:
                    if len(prod)>2 and "X" not in prod:
                        cfg_4[key].remove(prod)

                        if prod[:-1] in replaced.values():
                            value = get_key(prod[:-1], replaced)
                            cfg_4[key].append(value + prod[-1])
                            
                        else:
                            value = "X" + str(i) + prod[-1]
                            cfg_4[key].append(value)
                            cfg_4["X" + str(i)] = [prod[:-1]]
                            replaced["X" + str(i)] = prod[:-1]
                            i += 1
                            
            self.cfg = cfg_4.copy()
            return self.cfg
        
        # step 5 - remove terminals
        def remove_terminals():
            
            cfg_5 = copy.deepcopy(self.cfg)
            replaced_t = {}
            j = 0
            
            for key in self.cfg.keys():
                for prod in self.cfg[key]:
                    if (prod[0] in vt and prod[-1] not in vt):

                        cfg_5[key].remove(prod)

                        if prod[0] in replaced_t.keys():
                            cfg_5[key].append(replaced_t[prod[0]] + prod[-1])

                        else:
                            value = "Y" + str(j) + prod[-1]
                            cfg_5[key].append(value)
                            cfg_5["Y" + str(j)] = [prod[0]]
                            replaced_t[prod[0]] = "Y" + str(j)
                            j += 1

                    elif(prod[-1] in vt and prod[0] not in vt):

                        cfg_5[key].remove(prod)

                        if prod[-1] in replaced_t.keys():
                            cfg_5[key].append(replaced_t[prod[0] + replaced_t[prod[-1]]])

                        else:
                            value = prod[0] + "Y" + str(j)
                            cfg_5[key].append(value)
                            cfg_5["Y" + str(j)] = [prod[-1]]
                            replaced_t[prod[-1]] = "Y" + str(j)
                            j += 1
                            
            self.cfg = cfg_5.copy()
            return self.cfg
        
        check_start()
        print("Start symbol check:")
        print(self.cfg)
        
        remove_null(self.cfg)
        print("Null removal:")
        print(self.cfg)
        
        remove_unit(self.cfg)
        print("Unit productions removal:")
        print(self.cfg)
        
        remove_long_prod()
        print("More than 2 var productions removal:")
        print(self.cfg)
        
        remove_terminals()
        print("Remove terminals:")
        print(self.cfg)
        
        self.cnf = copy.deepcopy(self.cfg)

        return self.cnf

In [3]:
# function for scanning the .txt file
def scan_file(string):
    cfg = open(string)
    cfg = cfg.readlines()
    program = []
    for line in cfg:
        newline = line.strip()
        program.append(newline)
    return program

In [4]:
program = scan_file('cfg_input1.txt')

In [5]:
vn = ['S', 'A', 'B']
vt = ['a', 'b']

In [6]:
x = CFG(program, vt, vn)

In [7]:
x.parse_input()
x.cfg_2_cnf()

Start symbol check
{'S': ['ASA', 'aB'], 'A': ['B', 'S'], 'B': ['b', 'eps'], 'S0': ['S']}
Null removal
{'S': ['ASA', 'aB', 'a', 'SA', 'AS', 'S'], 'A': ['B', 'S'], 'B': ['b'], 'S0': ['S']}
Unit productions removal
{'S': ['ASA', 'aB', 'a', 'SA', 'AS'], 'A': ['b', 'ASA', 'aB', 'a', 'SA', 'AS'], 'B': ['b'], 'S0': ['ASA', 'aB', 'a', 'SA', 'AS']}
More than 2 var productions removal
{'S': ['aB', 'a', 'SA', 'AS', 'X0A'], 'A': ['b', 'aB', 'a', 'SA', 'AS', 'X0A'], 'B': ['b'], 'S0': ['aB', 'a', 'SA', 'AS', 'X0A'], 'X0': ['AS']}
Remove terminals
{'S': ['a', 'SA', 'AS', 'X0A', 'Y0B'], 'A': ['b', 'a', 'SA', 'AS', 'X0A', 'Y0B'], 'B': ['b'], 'S0': ['a', 'SA', 'AS', 'X0A', 'Y0B'], 'X0': ['AS'], 'Y0': ['a']}


{'S': ['a', 'SA', 'AS', 'X0A', 'Y0B'],
 'A': ['b', 'a', 'SA', 'AS', 'X0A', 'Y0B'],
 'B': ['b'],
 'S0': ['a', 'SA', 'AS', 'X0A', 'Y0B'],
 'X0': ['AS'],
 'Y0': ['a']}

In [8]:
program = scan_file('cfg_input.txt')

In [9]:
vn = ['S', 'D', 'E', 'F', 'L']
vt = ['a', 'b', 'c', 'd']

In [10]:
x = CFG(program, vt, vn)
x.parse_input()
x.cfg_2_cnf()

Start symbol check
{'S': ['aD'], 'D': ['bE'], 'E': ['cF', 'dL'], 'F': ['dD'], 'L': ['aL', 'bL', 'c']}
Null removal
{'S': ['aD'], 'D': ['bE'], 'E': ['cF', 'dL'], 'F': ['dD'], 'L': ['aL', 'bL', 'c']}
Unit productions removal
{'S': ['aD'], 'D': ['bE'], 'E': ['cF', 'dL'], 'F': ['dD'], 'L': ['aL', 'bL', 'c']}
More than 2 var productions removal
{'S': ['aD'], 'D': ['bE'], 'E': ['cF', 'dL'], 'F': ['dD'], 'L': ['aL', 'bL', 'c']}
Remove terminals
{'S': ['Y0D'], 'D': ['Y1E'], 'E': ['Y2F', 'Y3L'], 'F': ['Y3D'], 'L': ['c', 'Y0L', 'Y1L'], 'Y0': ['a'], 'Y1': ['b'], 'Y2': ['c'], 'Y3': ['d']}


{'S': ['Y0D'],
 'D': ['Y1E'],
 'E': ['Y2F', 'Y3L'],
 'F': ['Y3D'],
 'L': ['c', 'Y0L', 'Y1L'],
 'Y0': ['a'],
 'Y1': ['b'],
 'Y2': ['c'],
 'Y3': ['d']}